In [ ]:
#default_exp explorer

In [ ]:
#export
from fastai2.vision.all import *
from fastexplorer.representation import *
from starlette.applications import Starlette
from starlette.endpoints import WebSocketEndpoint
from starlette.websockets import WebSocketDisconnect
import uvicorn, asyncio, json, logging
import nest_asyncio
nest_asyncio.apply()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False)

# Fast Explorer

> Here we will setup the server and store the model representation on the fastai `Learn` object..

## Event types

In [ ]:
#export
class Events:
    def __init__(self, d):
        for k,v in d.items(): setattr(self, k, v)
            
    def __repr__(self):
        d = '\n'.join([f'  {k}: {v}' for k,v in self.__dict__.items()])
        res = f'{self.__class__.__name__} (\n{d}\n)'
        return res

clientEvents = Events({
    'SEND_DATA': 'socket/socketReceiveData',
    'INVALID_EVENT': 'socket/socketInvalidEvent',
    'SEND_IMAGE_INPUT': 'socket/socketReceiveImageInput',
    'CLOSE_CLIENT': 'socket/socketServerClosed',
})

serverEvents = Events({
    'RECEIVE_EVENT': 'socket/sendEvent',
})

**Available events**

clientEvents define the actions managed by the web client:

In [ ]:
clientEvents

Events (
  SEND_DATA: socket/socketReceiveData
  INVALID_EVENT: socket/socketInvalidEvent
  SEND_IMAGE_INPUT: socket/socketReceiveImageInput
  CLOSE_CLIENT: socket/socketServerClosed
)

serverEvents define the actions managed by the `FastExplorer` server.

In [ ]:
serverEvents

Events (
  RECEIVE_EVENT: socket/sendEvent
)

## Server

In [ ]:
#export
class FastExplorer:
    'Wrapper around `Representation` and `ProxyServer`.'
    def __init__(self, learn, host='0.0.0.0', port=8000):
        store_attr(self, 'learn,host,port')
        self.representation = learn.to_representation()
        self.server = Starlette()
        self.endpoint = self.server.websocket_route('/ws')(self.endpoint)
        self.socket = None
        
    def __repr__(self): return f'{self.__class__.__name__} ()'

    async def handle_web_client(self, websocket):
        while True:
            try:
                msg = await websocket.receive_json()
                logging.info(f'Client msg: {msg}')
                await self.dispatch_event(websocket, msg)
            except json.JSONDecodeError: logging.info('Invalid JSON received')
            except WebSocketDisconnect: break
    
    async def dispatch_event(self, websocket, msg):
        typ = msg['type']
        event = msg['payload']['event'].lower()
        
        if typ != serverEvents.RECEIVE_EVENT:
            logging.info(f'Invalid type: {typ!r}')
        else:
            if hasattr(self, event):
                logging.info(f'Launching: {event}')
                await getattr(self, event)(websocket)
            else:
                logging.info(f'Invalid event: {event!r}')
                await websocket.send_json({
                    'type': clientEvents.INVALID_EVENT,
                    'payload': {'type': typ}
                })

    async def endpoint(self, websocket):
        # Open socket
        await websocket.accept()
        msg = await websocket.receive_json()
        self.socket = websocket
        logging.info(f'Client connected: {msg}')
        
        # Manage requests
        await websocket.send_json({
                'type': clientEvents.SEND_DATA,
                'payload': { 'data': self.representation.to_json()}
            })
        await self.handle_web_client(websocket)
        
        # Close socket
        await self.close_client(websocket, 'Finalized by server')
        logging.info(f'Client closed: {msg}')    
    
    async def close_client(self, websocket, msg):
        await websocket.send_json({
            'type': clientEvents.CLOSE_CLIENT,
            'payload': {'msg': msg}
        })
        await websocket.close()
    
    def serve(self): uvicorn.run(self.server, host=self.host, port=self.port)

In [ ]:
#export
@patch
@delegates(FastExplorer.__init__)
def fastexplorer(self:Learner, reload=False, **kwargs):
    if (not hasattr(self, 'explorer')) or reload: self.explorer = FastExplorer(self, **kwargs)
    self.explorer.serve()

## Utils

In [ ]:
#export
def header_data_from_array_1_0(array, typ):
    d = {'shape': array.shape}
    if array.flags.c_contiguous:
        d['fortran_order'] = False
    elif array.flags.f_contiguous:
        d['fortran_order'] = True
    else:
        d['fortran_order'] = False

    d['descr'] = np.lib.format.dtype_to_descr(array.dtype)
    d['type'] = typ
    return d

def _write_array_header(d, version=None):
    header = ["{"]
    for key, value in sorted(d.items()):
        # Need to use repr here, since we eval these when reading
        header.append("'%s': %s, " % (key, repr(value)))
    header.append("}")
    header = "".join(header)
    header = np.lib.format._filter_header(header)
    if version is None:
        header = np.lib.format._wrap_header_guess_version(header)
    else:
        header = np.lib.format._wrap_header(header, version)
    return header

def _get_numpy_bytes(x, typ):
    a = _write_array_header(header_data_from_array_1_0(xx, typ))
    b = x.tobytes()
    return a+b

In [ ]:
xb,yb = dls.one_batch()
xx = xb.cpu().numpy()[0,0]
xx.shape

(28, 28)

## Events handlers

> Functions defined here are sended by the web client in upper case (eg: LOAD_INPUT)

In [ ]:
#export
@patch
async def load_input(self:FastExplorer, websocket):
    "Sends a data sample to the client."
    xb,yb = self.learn.dls.one_batch()
    # random
    idx = np.random.choice(xb.size(0))
    x = xb[idx,0].cpu().numpy() # TODO
    array_bytes = _get_numpy_bytes(x, clientEvents.SEND_IMAGE_INPUT)
    await websocket.send_bytes(array_bytes)

In [ ]:
show_doc(FastExplorer.load_input)

<h4 id="FastExplorer.load_input" class="doc_header"><code>FastExplorer.load_input</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>FastExplorer.load_input</code>(**`websocket`**)

Sends a data sample to the client.

In [ ]:
#srv
learn.fastexplorer(True)

INFO:     Started server process [11441]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 60692) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'type': 'CONNECTED', 'payload': {'client': 'web_client'}}
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': {'event': 'LOAD_INPUT'}}
INFO:     Launching: load_input
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': {'event': 'LOAD_INPUT'}}
INFO:     Launching: load_input
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': {'event': 'LOAD_INPUT'}}
INFO:     Launching: load_input
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': {'event': 'LOAD_INPUT'}}
INFO:     Launching: load_input
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': {'event': 'LOAD_INPUT'}}
INFO:     Launching: load_input
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': 

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_representation.ipynb.
Converted 01_explorer.ipynb.
Converted 10_tutorial.ipynb.
Converted index.ipynb.
